In [1]:
# importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
import re

In [2]:
# cargar datos
df_abc = pd.read_csv('files/hr.csv')

## ANÁLISIS DE ATTRITION (Exploratorio + Interpretativo)

### 1. Distribución de Attrition

En este apartado se analiza la distribución de la variable Attrition con el objetivo de comprender el nivel de rotación de empleados en el conjunto de datos.

In [6]:
df_abc.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [7]:
df_abc['Attrition'].value_counts()

Attrition
No     1236
Yes     238
Name: count, dtype: int64

In [9]:
df_abc['Attrition'].value_counts(normalize=True) * 100

Attrition
No     83.85346
Yes    16.14654
Name: proportion, dtype: float64

In [ ]:
# Visualización

df_abc['Attrition'].value_counts().plot(kind='bar')

plt.figure(figsize=(6, 4))   # estandarición tamaños
plt.title('Distribución de Attrition')
plt.xlabel('Attrition')
plt.ylabel('Número de empleados')

# Guardar la figura
plt.savefig('../outputs/distribucion_attrition.png', bbox_inches='tight')

# Mostrar el gráfico
plt.show()

NameError: name 'df_abc' is not defined

### OBSERVACIONES sobre la DISTRIBUCION  de Attrition

La mayoría de los empleados del conjunto de datos continúa en la empresa. 
Aproximadamente el 83.85% de los registros corresponden a empleados que permanecen en la compañía, mientras que solo el 16.15% presentan Attrition.

Esto indica que el conjunto de datos está claramente desbalanceado, con una proporción significativamente mayor de empleados que no han dejado la empresa.
Este desbalance deberá tenerse en cuenta en los análisis posteriores, ya que puede influir en la interpretación de los resultados y en la identificación de patrones asociados a la rotación.

Este desbalance es habitual en datasets de recursos humanos, donde la rotación suele afectar a una minoría de empleados.